<a href="https://colab.research.google.com/github/Yphy/NLP/blob/master/dart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
import json
import time

In [15]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
api_code = '1ad5e3556e6de0c6e73c59883409f908364e8f25'
crp_code = ''
start_date = '19900101' 
url = 'http://dart.fss.or.kr/api/search.json?auth='+api_code+'&crp_cd='+crp_code+'&start_dt='+start_date+'&bsn_tp=A001'

In [0]:
def get_df(code): #검색된 회사의 사업보고서 내용을 데이터프레임으로 저장
    api_code = '1ad5e3556e6de0c6e73c59883409f908364e8f25'
    start_date = '19900101'
    url = 'http://dart.fss.or.kr/api/search.json?auth='+api_code+'&crp_cd='+code+'&start_dt='+start_date+'&bsn_tp=A001'
    req =requests.get(url)
    html = req.text
    code = json.loads(html)
    soup = BeautifulSoup(html,'html.parser')
    
    data = pd.DataFrame() 
    for t in code['list']:
        temp = pd.DataFrame(([[t['crp_cls'],t['crp_nm'],t['crp_cd'],t['rpt_nm'],t['rcp_no'],t['flr_nm'],t['rcp_dt'],t['rmk']]])
                            ,columns=["crp_cls","crp_nm","crp_cd","rpt_nm","rcp_no","flr_nm","rcp_dt","rmk"])
        data = pd.concat([data,temp])
    data.reset_index(drop=True,inplace=True)
    return(data)


In [0]:
def get_url(data): #dataframe을 넣었을 때 해당 사업보고서의 주소를 저장
    rcp_ls=[]
    for row in data['rcp_no']:
        rcp_ls.append(row)
        urls = []
        for i in rcp_ls:
            urls.append('https://dart.fss.or.kr/dsaf001/main.do?rcpNo='+i)
    
    return urls

In [0]:
def final_url(url):
    try:
        req2 = requests.get(url)
        html2 = req2.text
        soup2 = BeautifulSoup(html2, 'html.parser')
        body = str(soup2.find('head'))

        a = re.search('이사의 경영진단 및 분석의견',body).span()
        b= re.search(r'viewDoc(.*);',body[a[0]:]).group()
        list = b[8:-2].split(',')
        list = [i[1:-1] for i in list]
        list[1] = list[1][1:] #드러움
        list[2] = list[2][1:]
        list[3] = list[3][1:]
        list[4] = list[4][1:]
        list[5] = list[5][1:]    

        url_final = 'http://dart.fss.or.kr/report/viewer.do?rcpNo='+ list[0] + '&dcmNo='+ list[1]+'&eleId=' +list[2] + '&offset=' + list[3] + '&length=' + list[4] + '&dtd=dart3.xsd'
        return(url_final)
    
    except AttributeError as e:
        print(e)

In [0]:
def extracting_text(url):
    req3 = requests.get(url)
    html3 = req3.text
    soup3 = BeautifulSoup(html3,'html.parser')
    tables = []
    ka = soup3.find_all('span') #table을 제외한 본문 text 부분 찾아서 리스트안에저장
    for k in ka:
        tables.append(k.get_text())
    table = ''.join(tables) #리스트를 string으로 바꾼다
    table = table.replace('\xa0','') #정리
    return table
    

In [0]:
def extracting_text2(url):
    req3 = requests.get(url)
    html3 = req3.text
    soup3 = BeautifulSoup(html3,'html.parser')
    tables = []
    ka = soup3.find_all('p') #table을 제외한 본문 text 부분 찾아서 리스트안에저장
    for k in ka:
        tables.append(k.get_text())
    table = ''.join(tables) #리스트를 string으로 바꾼다
    table = table.replace('\xa0','') #정리
    table = table.replace('\n','')
    return table

In [0]:
def final(crp_code):
    try:
        df = get_df(crp_code)
        urls = get_url(df)
        strs=[]
        for i in urls:
            strs.append(extracting_text2(final_url(i)))
        str_series = pd.Series(strs)
        df['str']= str_series
        return(df)
    
    except:
        pass

In [0]:
def make_df(df):
    start_vect=time.time()
    ls = []
    body = pd.DataFrame()
    try:
        for i in df['Symbol']:
            ls.append(str(i).zfill(6))
        for i in ls:
                temp = final(i)
                body = pd.concat([body,temp],ignore_index=True)
    except AttributeError as e:
        print(e)
    
    print("training Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))
    return(body)



In [18]:
kospi_df = pd.read_csv('/gdrive/My Drive/colab/dart/코스피상장회사.csv',index_col=0) 

FileNotFoundError: ignored

In [0]:
!cd gdrive/My\ Drive/colab

In [27]:
!ls gdrive/My\ Drive/colab/dart

코스피상장회사.csv  df_full.csv  full_df.csv  kospi200.csv


In [0]:
book = pd.read_csv('gdrive/My Drive/colab/dart/kospi200.csv') #kospi200 기업 데이터 불러오기

In [0]:
book1 = book[0:50]
book2 = book[50:100]
book3 = book[100:150]
book4 = book[150:200]

In [0]:
df_1 = make_df(book1) #kospi200기업 part1을 저장 7nan

'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
training Runtime: 13.93 Minutes


In [0]:
df_2 = make_df(book2) #part2 :6nan

'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
training Runtime: 13.32 Minutes


In [0]:
df_3 = make_df(book3) #part3:6nan ,part4:4nan
df_4 = make_df(book4)

'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
training Runtime: 13.01 Minutes
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
'NoneType' object has no attribute 'span'
training Runtime: 12.83 Minutes


In [0]:
df_full = pd.concat([df_1,df_2,df_3,df_4],ignore_index=True)

In [0]:
df_full.to_csv('/gdrive/My Drive/colab/dart/full_df.csv') #csv파일 저장

In [0]:
full_df = pd.read_csv('gdrive/My Drive/colab/dart/full_df.csv') #test